In [1]:
import ee
import collections
collections.Callable = collections.abc.Callable
import geemap
from geemap import chart

%matplotlib inline
import matplotlib.pyplot as plt

import math

#ee.Authenticate()
ee.Initialize()

### Imports and Constants

In [13]:
Map = geemap.Map()

# Emily's Regions
region1 = ee.Geometry.Polygon(
        [[[25.75, -10.75],
          [25.75, -10.8],
          [25.9, -10.8],
          [25.9, -10.75]]])
region2 = ee.Geometry.Polygon(
        [[[25.85, -10.6],
          [25.85, -10.65],
          [25.95, -10.65],
          [25.95, -10.6]]])
region3 = ee.Geometry.Polygon(
        [[[26, -10.5],
          [26, -10.75],
          [26.25, -10.75],
          [26.25, -10.5]]])
region4 = ee.Geometry.Polygon(
        [[[25.95, -10.6],
          [25.95, -10.7],
          [26.05, -10.7],
          [26.05, -10.6]]])
noMine = ee.Geometry.Polygon(
        [[[26.05, -10.7],
          [26.05, -10.8],
          [26.15, -10.8],
          [26.15, -10.7]]])
total = ee.Geometry.Polygon(
        [[[25.8, -10.5],
          [25.8, -10.8],
          [26.1, -10.8],
          [26.1, -10.5]]])

# Rishi's Area
rishiArea = ee.Geometry.Polygon(
        [[[27.35, -7.5],
          [27.35, -7.6],
          [27.45, -7.6],
          [27.45, -7.5]]])

# Ray's Areas 
roi1 = ee.Geometry.Polygon(
        [[[29.55, 3.16],
          [29.55, 3.1],
          [29.63, 3.1],
          [29.63, 3.16]]])
roi2 = ee.Geometry.Polygon(
        [[[29.76, 3.16],
          [29.76, 3.12],
          [29.8, 3.12],
          [29.8, 3.16]]])
roi3 = ee.Geometry.Polygon(
        [[[30.25, 1.8],
          [30.25, 1.7],
          [30.35, 1.7],
          [30.35, 1.8]]])

# Data Sets  Sentinel-2 and Landsat 5
s2 = ee.ImageCollection('COPERNICUS/S2_SR')
ls5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')

# Matching Bands
bandsLS5 = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']
bandsS2 = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
bandNames = ['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2']

# Visualization
rgbVis = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['R', 'G', 'B'],
}

In [14]:
# mask clouds and scale bands for LS5
def maskL457sr(image):
  # Bit 0 - Fill
  # Bit 1 - Dilated Cloud
  # Bit 2 - Unused
  # Bit 3 - Cloud
  # Bit 4 - Cloud Shadow
  #qaMask = image.select('QA_PIXEL').bitwiseAnd(parseInt('11111', 2)).eq(0)
  qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
  saturationMask = image.select('QA_RADSAT').eq(0)

  # Apply the scaling factors to the appropriate bands.
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)

  # Replace the original bands with the scaled ones and apply the masks.
  return image.addBands(opticalBands, None, True) \
      .addBands(thermalBand, None, True) \
      .updateMask(qaMask) \
      .updateMask(saturationMask)

# Function to mask clouds using the Sentinel-2 QA band
def maskS2clouds(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0))

  return image.updateMask(mask).divide(10000)

### Classification

In [15]:
# Training Data - Used on Total region for Sentinel-2, 2020 composite
# Selected based on RGB median composite visualized

nonVeg2020 = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([25.884051513674073, -10.612839387311823]),
            {
              "landcover": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([25.809550476076417, -10.77881944655002]),
            {
              "landcover": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([25.86070556640845, -10.781517557036542]),
            {
              "landcover": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([25.8909179687522, -10.662103099284176]),
            {
              "landcover": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([25.9788085937522, -10.64185870124958]),
            {
              "landcover": 1,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([26.06500815525748, -10.63246528279504]),
            {
              "landcover": 1,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([25.932501220912915, -10.613290891172372]),
            {
              "landcover": 1,
              "system:index": "6"
            })])
veg2020 = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([25.971598815920167, -10.63713481487796]),
            {
              "landcover": 2,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([25.828776550295167, -10.650968846848661]),
            {
              "landcover": 2,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([26.023783874513917, -10.737332956336642]),
            {
              "landcover": 2,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([25.8689453125022, -10.767689485172806]),
            {
              "landcover": 2,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([25.88439483642798, -10.790286248993706]),
            {
              "landcover": 2,
              "system:index": "4"
            }),
        ee.Feature(
            ee.Geometry.Point([25.814382544905918, -10.56429718435804]),
            {
              "landcover": 2,
              "system:index": "5"
            }),
        ee.Feature(
            ee.Geometry.Point([26.063634864241855, -10.602770155202851]),
            {
              "landcover": 2,
              "system:index": "6"
            }),
        ee.Feature(
            ee.Geometry.Point([26.077367774398105, -10.745820038885466]),
            {
              "landcover": 2,
              "system:index": "7"
            })])

# Classification
training = veg2020.merge(nonVeg2020)

# Image for Training Data
ref2020 = s2 \
                .filter(ee.Filter.bounds(total)) \
                .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                .map(maskS2clouds) \
                .select(bandsS2, bandNames) \
                .median().clip(total)

# Overlay the point on the image to get training data.
training = ref2020.sampleRegions(**{
  'collection': training,
  'properties': ['landcover'],
  'scale': 10
})

# Train a classifier.
classifier = ee.Classifier.smileRandomForest(50).train(**{
  'features': training,
  'classProperty': 'landcover',
  'inputProperties': ref2020.bandNames()
})

### Select Desired Region

In [16]:
mainRegion = total

### Reference Layers

In [17]:
image2000 = ls5 \
                .filter(ee.Filter.bounds(mainRegion)) \
                .filter(ee.Filter.date('2000-01-01', '2000-12-31')) \
                .map(maskL457sr) \
                .select(bandsLS5, bandNames) \
                .median().clip(mainRegion)

image2020 = s2 \
                .filter(ee.Filter.bounds(mainRegion)) \
                .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
                .map(maskS2clouds) \
                .select(bandsS2, bandNames) \
                .median().clip(mainRegion)

Map.centerObject(mainRegion, 11)

Map.addLayer(image2000, rgbVis, 'RGB 2000 (LS5)')
Map.addLayer(image2020, rgbVis, 'RGB 2020 (S2)')
# classified1 = image2000.classify(classifier)
# classified2 = image2020.classify(classifier)
# Map.addLayer(classified1, {'min': 1, 'max': 2, 'palette': ['red', 'green']}, '2000_classified', False)
# Map.addLayer(classified2, {'min': 1, 'max': 2, 'palette': ['red', 'green']}, '2020_classified', False)

### Create Square Regions

In [18]:
"""
Segment the given geometry into squares of given size (in km)
:param geometry: rectangle form geometry object
:param size: desired square dimension in km
:return: list including all squares
"""
def create_segments(geometry, size):
    segments = []
    r_earth, dy, dx, pi = ee.Number(6378), ee.Number(size), ee.Number(size), ee.Number(math.pi)
    
    coords = ee.List(geometry.coordinates().get(0)).slice(0, -1)
    
    top = ee.Number(ee.List(coords.get(2)).get(1))
    left = ee.Number(ee.List(coords.get(0)).get(0))
    
    width = int(ee.Geometry.Point(coords.get(0)).distance(ee.Geometry.Point(coords.get(1))).divide(1000 * size).getInfo())
    height = int(ee.Geometry.Point(coords.get(1)).distance(ee.Geometry.Point(coords.get(2))).divide(1000 * size).getInfo())

    for y in range(height + 1):
        left = ee.Number(ee.List(coords.get(0)).get(0))
        for x in range(width + 1):
            #
            first = top
            second = dx.divide(r_earth)
            third = ee.Number(180).divide(pi)
            con = pi.divide(ee.Number(180))
            fourth = left.multiply(con).multiply(con).cos()
            
            new_lon = first.subtract(second.multiply(third).divide(fourth))
            #new_lon = top - (dx / r_earth) * (180 / pi) / math.cos(math.radians(left * pi/180))
            #new_lat = left  + (dy / r_earth) * (180 / pi)
            new_lat = left.add((dy.divide(r_earth)).multiply((ee.Number(180).divide(pi))))
            
            square = ee.Geometry.Polygon(
                [[[left, new_lon],
                  [new_lat, new_lon],
                  [new_lat, top],
                  [left, top]]], None, False)
            
            segments.append(square)
            
            left = new_lat
        top = new_lon
        
    return segments

In [19]:
regions = create_segments(mainRegion, 1)
print('Number of Regions Created: ' + str(len(regions)))

Number of Regions Created: 1122


### Function to Convert Object to a Geometry

In [20]:
def coordList(feat):
    return ee.Feature(feat).geometry()

### Percent Vegetation Loss

In [21]:
def vegLoss(region):
    # Images
    image2000 = ls5 \
        .filter(ee.Filter.bounds(region)) \
        .filter(ee.Filter.date('2000-01-01', '2000-12-31')) \
        .map(maskL457sr) \
        .select(bandsLS5, bandNames) \
        .median().clip(region)
    image2020 = s2 \
        .filter(ee.Filter.bounds(region)) \
        .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(maskS2clouds) \
        .select(bandsS2, bandNames) \
        .median().clip(region)
    
    # Classify the images
    classified2020 = image2020.classify(classifier)
    classified2000 = image2000.classify(classifier)

    # Area of Vegetation for each image
    veg1 = classified2000.eq(2)
    veg1Image = veg1.multiply(ee.Image.pixelArea())
    area1 = veg1Image.reduceRegion(**{
      'reducer': ee.Reducer.sum(),
      'geometry': region,
      'scale': 30,
      'maxPixels': 1e10
      })
    veg1SqKm = ee.Number(area1.get('classification')).divide(1e6)

    veg2 = classified2020.eq(2)
    veg2Image = veg2.multiply(ee.Image.pixelArea())
    area2 = veg2Image.reduceRegion(**{
      'reducer': ee.Reducer.sum(),
      'geometry': region,
      'scale': 30,
      'maxPixels': 1e10
      })
    veg2SqKm = ee.Number(area2.get('classification')).divide(1e6)

    # Vegetation Loss: Greater than 20%?
    status = ((veg1SqKm.eq(0).And(veg2SqKm.eq(0))).Or(((veg1SqKm.subtract(veg2SqKm)).divide(veg1SqKm)).gt(0.2)))
    reg = ee.Feature(ee.Geometry(region), {'status': status})
    return reg

# Filter list of regions for vegetation loss > 20%
status_list = ee.List(regions).map(vegLoss)
passed_list = status_list.filter(ee.Filter.eq('status',1))

# Add passing regions to the Map (using Sentinel-2 imagery)
passed_geo = passed_list.map(coordList)
layer_veg = ee.Geometry.MultiPolygon(passed_geo)

image_veg = s2 \
    .filter(ee.Filter.bounds(layer_veg)) \
    .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(maskS2clouds) \
    .select(bandsS2, bandNames) \
    .median().clip(layer_veg)
Map.addLayer(image_veg, rgbVis, 'pass vegetation')

### NDMI Loss

In [22]:
def ndmiLoss(region):    
    # Images
    image2000 = ls5 \
        .filter(ee.Filter.bounds(region)) \
        .filter(ee.Filter.date('2000-01-01', '2000-12-31')) \
        .map(maskL457sr) \
        .select(bandsLS5, bandNames) \
        .median().clip(region)
    ndmi2000 = image2000.normalizedDifference(['NIR', 'SWIR1']).rename('NDMI')
    image2020 = s2 \
        .filter(ee.Filter.bounds(region)) \
        .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(maskS2clouds) \
        .select(bandsS2, bandNames) \
        .median().clip(region)
    ndmi2020 = image2020.normalizedDifference(['NIR', 'SWIR1']).rename('NDMI')
    
    # Average NDMI
    stats2000 = ndmi2000.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': ndmi2000.geometry(),
        'scale': 30
    })    
    stats2020 = ndmi2020.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': ndmi2020.geometry(),
        'scale': 30
    })
    
    avg2000 = ee.Number(stats2000.get('NDMI'))
    avg2020 = ee.Number(stats2020.get('NDMI'))
    
    # NDMI Loss?
    status = avg2000.subtract(avg2020).gte(0)
    
    reg = ee.Feature(ee.Geometry(region), {'status': status})
    return reg

# Filter list of regions for NDMI loss
status_list_NDMI = ee.List(passed_geo).map(ndmiLoss)
passed_list_NDMI = status_list_NDMI.filter(ee.Filter.eq('status',1))

# Add passing regions to the Map (using Sentinel-2 imagery)
passed_geo_NDMI = passed_list_NDMI.map(coordList)
layer_NDMI = ee.Geometry.MultiPolygon(passed_geo_NDMI)
image_NDMI = s2 \
    .filter(ee.Filter.bounds(layer_NDMI)) \
    .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(maskS2clouds) \
    .select(bandsS2, bandNames) \
    .median().clip(layer_NDMI)
Map.addLayer(image_NDMI, rgbVis, 'pass NDMI')

### NIR/SWIR2 Loss

In [23]:
def ns2Loss(region):
    # Images
    image2000 = ls5 \
        .filter(ee.Filter.bounds(region)) \
        .filter(ee.Filter.date('2000-01-01', '2000-12-31')) \
        .map(maskL457sr) \
        .select(bandsLS5, bandNames) \
        .median().clip(region)
    ns2000 = image2000.normalizedDifference(['NIR', 'SWIR2']).rename('nirSwir2')
    image2020 = s2 \
        .filter(ee.Filter.bounds(region)) \
        .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(maskS2clouds) \
        .select(bandsS2, bandNames) \
        .median().clip(region)
    ns2020 = image2020.normalizedDifference(['NIR', 'SWIR2']).rename('nirSwir2')
    
    # Average NIR/SWIR2
    stats2000 = ns2000.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': ns2000.geometry(),
        'scale': 30
    })    
    stats2020 = ns2020.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': ns2020.geometry(),
        'scale': 30
    })
    
    avg2000 = ee.Number(stats2000.get('nirSwir2'))
    avg2020 = ee.Number(stats2020.get('nirSwir2'))
    
    # NIR/SWIR2 Loss?
    status = avg2000.subtract(avg2020).gte(0)
    
    reg = ee.Feature(ee.Geometry(region), {'status': status})
    return reg

# Filter list of regions for NIR/SWIR2 loss
status_list_NS2 = ee.List(passed_geo_NDMI).map(ns2Loss)
passed_list_NS2 = status_list_NS2.filter(ee.Filter.eq('status',1))

# Add passing regions to the Map (using Sentinel-2 imagery)
passed_geo_NS2 = passed_list_NS2.map(coordList)
layer_NS2 = ee.Geometry.MultiPolygon(passed_geo_NS2)
image_NS2 = s2 \
    .filter(ee.Filter.bounds(layer_NS2)) \
    .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(maskS2clouds) \
    .select(bandsS2, bandNames) \
    .median().clip(layer_NS2)
Map.addLayer(image_NS2, rgbVis, 'pass NIR/SWIR2')

### NIR/G Loss

In [24]:
def ngLoss(region):
    # Images
    image2000 = ls5 \
        .filter(ee.Filter.bounds(region)) \
        .filter(ee.Filter.date('2000-01-01', '2000-12-31')) \
        .map(maskL457sr) \
        .select(bandsLS5, bandNames) \
        .median().clip(region)
    ng2000 = image2000.normalizedDifference(['NIR', 'G']).rename('nirG')
    image2020 = s2 \
        .filter(ee.Filter.bounds(region)) \
        .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(maskS2clouds) \
        .select(bandsS2, bandNames) \
        .median().clip(region)
    ng2020 = image2020.normalizedDifference(['NIR', 'G']).rename('nirG')
    
    # Average NIR/G
    stats2000 = ng2000.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': ng2000.geometry(),
        'scale': 30
    })    
    stats2020 = ng2020.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': ng2020.geometry(),
        'scale': 30
    })
    
    avg2000 = ee.Number(stats2000.get('nirG'))
    avg2020 = ee.Number(stats2020.get('nirG'))
    
    # NIR/G Loss: Greater than .02?
    status = avg2000.subtract(avg2020).gte(0.02)
    
    reg = ee.Feature(ee.Geometry(region), {'status': status})
    return reg

# Filter list of regions for NIR/G loss > .02
status_list_NG = ee.List(passed_geo_NS2).map(ngLoss)
passed_list_NG = status_list_NG.filter(ee.Filter.eq('status',1))

# Add passing regions to the Map (using Sentinel-2 imagery)
passed_geo_NG = passed_list_NG.map(coordList)
layer_NG = ee.Geometry.MultiPolygon(passed_geo_NG)
image_NG = s2 \
    .filter(ee.Filter.bounds(layer_NG)) \
    .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .map(maskS2clouds) \
    .select(bandsS2, bandNames) \
    .median().clip(layer_NG)
Map.addLayer(image_NG, rgbVis, 'pass NIR/G')

### MAP

In [25]:
Map

Map(center=[-10.650011046204405, 25.949999999999996], controls=(WidgetControl(options=['position', 'transparen…